In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [3]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
from sklearn.preprocessing import StandardScaler

series = pd.DataFrame({'1': pd.Series(np.random.normal(loc=0, scale=1, size=10)), 
                        '2': pd.Series(np.random.normal(loc=0, scale=1, size=10))})

# series = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=10)), 
#                         '2': pd.Series(np.arange(start=50, stop=60))})

exog = pd.DataFrame({'e1': pd.Series(np.arange(start=0, stop=10)), 
                        'e2': pd.Series(np.arange(start=50, stop=60))})

exog_predict = pd.DataFrame({'e1': pd.Series(np.arange(start=10, stop=15)),
                                'e2': pd.Series(np.arange(start=60, stop=65))})
exog_predict.index = pd.RangeIndex(start=10, stop=15)

In [5]:
# Custom function to create predictors
# ==============================================================================
def create_predictors(y):
    """
    Create first 10 lags of a time series.
    Calculate moving average with window 20.
    """

    lags = y[-1:-4:-1]   # window size needed = 3
    mean = np.mean(y[0]) # window size needed = 3
    predictors = np.hstack([lags, mean])

    return predictors

# Esto falla porque el check de _create_train_X_y_single_series recibe y_values[:-1]
# Entonces y[0] para el es el primer valor de la series, mientras que el usuario
# espera que sea el primer valor de last_window.

# En realidad no es un error, pero nosotros siempre esperamos que el usuario
# trabaje con índices negativos

forecaster = ForecasterAutoregMultiSeriesCustom(
    regressor          = LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    fun_predictors     = create_predictors,
    window_size        = 3,
    encoding           = None,
    dropna_from_series = False,
    differentiation    = 1,
    # transformer_series = StandardScaler(),
    # transformer_exog   = StandardScaler()
)
forecaster.fit(
    series=series, suppress_warnings=True
)

ValueError: The `window_size` argument (3), declared when initializing the forecaster, does not correspond to the window used by `create_predictors`.

In [ ]:
forecaster.predict(
    steps=3, suppress_warnings=True,
    levels = '1'
)